In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = "C:/Users/Camer/OneDrive/desktop/College/Digital Tools/DigitalTools_project/research_project/src/data/"

In [17]:
BTC = pd.read_csv(path + "BTC-USD.csv")
BTC['Date'] = pd.to_datetime(BTC['Date'], format='%Y/%m/%d')
BTC = BTC.set_index('Date',drop=True)

Get daily range for data

In [4]:
BTC['Range'] = BTC.High - BTC.Low

Functions to get all the data before and after a particular date.
Ensure the data is in correct format - date and index.

In [5]:
def before(data,date,period):
    ind = data.index.get_loc(date)
    
    start = max(0,ind-period)
    
    return(data.iloc[start:ind])

def after(data,date,period):
    ind = data.index.get_loc(date)
    
    end = min(len(BTC),ind+period+1)
    
    return(data.iloc[ind:end]) #include the info on the date given

Moving Average

In [6]:
def MA(data,colname,period):
    data['MA_'+str(period)] = 0
    for ind in range(period-1,len(data)):
        start = ind - period + 1
        end = ind
        
        data['MA_'+str(period)].iloc[ind] = np.mean(data[colname].iloc[start:end])
    return

Rolling Volatiltity - idea is to look for spikes aruond the dates of interest

In [8]:
def Roll_Var(data,colname,period):
    data['Var_'+str(period)] = 0
    for ind in range(period-1,len(data)):
        start = ind - period + 1
        end = ind
        
        data['Var_'+str(period)].iloc[ind] = np.var(data[colname].iloc[start:end])
    return

Load in FED data and get the dates where the FED rate changes

In [9]:
FED = pd.read_csv(path + "FED_rate.csv")
FED['DATE'] = pd.to_datetime(FED['DATE'], format='%d/%m/%Y')
FED = FED.set_index('DATE',drop=True)

In [47]:
dates_change = []
values_change=[]

crate = FED.IOER[1]

for i in range(1,len(FED)):
    nrate = FED.IOER[i]
    
    if nrate != crate:
        dates_change.append(FED.index[i])
        values_change.append(nrate-crate)
    
    crate = nrate
    
    
dates_star = pd.DataFrame({"Dates":dates_change,"Change":values_change})

In [48]:
dates_star # dates of fed changes and the differnce

,Dates,Change
0,2016-12-15,0.25
1,2017-03-16,0.25
2,2017-06-15,0.25
3,2017-12-14,0.25
4,2018-03-22,0.25
5,2018-06-14,0.20
6,2018-09-27,0.25
7,2018-12-20,0.20
8,2019-05-02,-0.05
9,2019-08-01,-0.25


Now, we want to look at the changes in BTC around these dates. We will check if there is correlation between these changes and the n-day change in BTC. I will check for n=1,2,3,...,7.

In [12]:
CHG_DATES = dates_star.Dates

CHANGE = pd.DataFrame(index=CHG_DATES,columns=['FED_change','0day','1day','2day','3day','4day','5day','6day','7day'])

for d in CHG_DATES:
    btc0 = float(before(BTC['Adj Close'],d,1)) #close day before change
    
    btc1 = after(BTC['Adj Close'],d,7) #closes on each day after the change
    btc1 = btc1.to_numpy()
    
    CHANGE.loc[d,['0day','1day','2day','3day','4day','5day','6day','7day']] = btc1 - btc0
    
    
CHANGE['FED_change'] = list(dates_star.Change)

for col in CHANGE.columns:
    CHANGE[col] = pd.to_numeric(CHANGE[col])

In [13]:
CHANGE.corr()

,FED_change,0day,1day,2day,3day,4day,5day,6day,7day
FED_change,1.000000,-0.121775,-0.071150,-0.137565,-0.220316,-0.156568,-0.197437,-0.272994,-0.241836
0day,-0.121775,1.000000,0.839905,0.800050,0.801722,0.741039,0.724519,0.763272,0.729151
1day,-0.071150,0.839905,1.000000,0.960274,0.940703,0.925157,0.875687,0.783426,0.757515
2day,-0.137565,0.800050,0.960274,1.000000,0.935841,0.865569,0.812844,0.719470,0.676462
3day,-0.220316,0.801722,0.940703,0.935841,1.000000,0.960578,0.908720,0.826185,0.801751
4day,-0.156568,0.741039,0.925157,0.865569,0.960578,1.000000,0.965985,0.874655,0.851037
5day,-0.197437,0.724519,0.875687,0.812844,0.908720,0.965985,1.000000,0.956686,0.939305
6day,-0.272994,0.763272,0.783426,0.719470,0.826185,0.874655,0.956686,1.000000,0.975990
7day,-0.241836,0.729151,0.757515,0.676462,0.801751,0.851037,0.939305,0.975990,1.000000


There does not appear to be any signifigant relationships between the FED changes and any of the 0-7 days changes after the FED rate change occurs.
Next I will look at the moving averages

Add MA for periods of 10, 50 and 100 days

In [19]:
MA(BTC,'Adj Close',10)
MA(BTC,'Adj Close',50)
MA(BTC,'Adj Close',100)

C:\Users\Camer\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
MA10 = BTC['MA_10'][BTC.MA_10>0]
MA50 = BTC['MA_50'][BTC.MA_50>0]
MA100 = BTC['MA_100'][BTC.MA_100>0]

CI10 = [np.quantile(MA10,0.025),np.quantile(MA10,0.975)]
CI50 = [np.quantile(MA50,0.025),np.quantile(MA50,0.975)]
CI100 = [np.quantile(MA100,0.025),np.quantile(MA100,0.975)]

In [52]:
CHG_DATES = dates_star.Dates
n = len(dates_star)

MA_DATA = pd.DataFrame(index=CHG_DATES,columns=['10MA_after','in CI10','50MA_after','in CI50','100MA_after','in CI100'])

for d in CHG_DATES:
    period10 = after(BTC['Adj Close'],d,10)
    MA_DATA.loc[d,'10MA_after'] = period10.mean()
    if period10.mean() < CI10[1] and period10.mean() > CI10[0]:
        MA_DATA.loc[d,'in CI10'] = 1
    else:
        MA_DATA.loc[d,'in CI10'] = 0
        
    period50 = after(BTC['Adj Close'],d,50)
    MA_DATA.loc[d,'50MA_after'] = period50.mean()
    if period50.mean() < CI50[1] and period50.mean() > CI50[0]:
        MA_DATA.loc[d,'in CI50'] = 1
    else:
        MA_DATA.loc[d,'in CI50'] = 0
        
    period100 = after(BTC['Adj Close'],d,100)
    MA_DATA.loc[d,'100MA_after'] = period100.mean()
    if period100.mean() < CI100[1] and period100.mean() > CI10[0]:
        MA_DATA.loc[d,'in CI100'] = 1
    else:
        MA_DATA.loc[d,'in CI100'] = 0

In [53]:
MA_DATA

,10MA_after,in CI10,50MA_after,in CI50,100MA_after,in CI100
Dates,,,,,,
2016-12-15,832.159,1,907.1,1,1007.87,1
2017-03-16,1039.83,1,1182.86,1,1737.59,1
2017-06-15,2621.48,1,2562.62,1,3301.4,1
2017-12-14,16789.5,1,13934.8,1,11728.9,1
2018-03-22,7876.63,1,8304.66,1,7788.05,1
2018-06-14,6510.16,1,6917.66,1,6766.36,1
2018-09-27,6598.83,1,6428.51,1,5190.92,1
2018-12-20,3914.6,1,3714.32,1,3808.29,1
2019-05-02,6108.16,1,7841.82,1,9381.62,1


The above is not exactly a great way to check such a relationship, the idea is to check if after one of these FED changes happens, does the MA over a certain period differ signifigantly from its usual range? We see in the above, this does not occur, we could possibly lower the confidence level, since the MA is going to be a very smooth curve and it would unlikely to expect large jumps in it